# Mediapipe preprocessing

#### This notebook contains the mediapipe functions to detect and crop relevant regions from unsorted photographs. Please specify a folder (INPUT_FOLDER) which contains subfolders; one for each subject. The subfolders shall contain unsorted images.

In [ ]:
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
import os

In [ ]:
INPUT_FOLDER = "D:/derma2/"  # <-- these are the unsorted images
SAVE_DIR = "D:/testoutput/"  # <-- crops will be saved here

PLOT = False  # debugging

In [ ]:
all_imgs = glob(INPUT_FOLDER + "/*/*.JPG")
print("Individual images:", len(all_imgs))

In [ ]:
from scipy.spatial.transform import Rotation as R
from glob import glob
import math

def vec_length(v: np.array):
    return np.sqrt(sum(i**2 for i in v))

def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0: 
        return v
    return v / norm

def dotproduct(v1, v2):
    return sum((a*b) for a, b in zip(v1, v2))

def length(v):
    return math.sqrt(dotproduct(v, v))

def angle(v1, v2):
    return math.acos(dotproduct(v1, v2) / (length(v1) * length(v2)))

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh
    
def find_head(image):
    with mp_face_mesh.FaceMesh(min_detection_confidence=0.7, min_tracking_confidence=0.5) as face_mesh:
        drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
        
        results = face_mesh.process(image)
    return results
    
mp_pose = mp.solutions.pose    
def find_body(image):
    
    with mp_pose.Pose(static_image_mode=True, model_complexity=2, 
                      enable_segmentation=True, min_detection_confidence=0.5) as pose:
        results = pose.process(image)
    return results


def extract_landmarks(results, image):
    size, color = 100, "red"
    
    left_hüfte = results.pose_landmarks.landmark[23]
    right_hüfte = results.pose_landmarks.landmark[24]
    orientation_vektor =(np.array([left_hüfte.x, left_hüfte.y, left_hüfte.z]) - np.array([right_hüfte.x, right_hüfte.y, right_hüfte.z]))

    a = angle(orientation_vektor, [1, 0, 0])
    b = angle(orientation_vektor, [0, 0, 1])
    c = angle(orientation_vektor, [-1, 0, 0])
    d = angle(orientation_vektor, [0, 0, -1])
   # print("Landmarks found at position:", np.argmin([a,b,c,d]), 
   #       list(["front", "right", "back", "left"])[np.argmin([a,b,c,d])])
    
    for f in range(32):
        if(f in [23,24,12]):
            size, color = 100, "red"
        else:
            size, color = 30, "yellow"

        x = results.pose_landmarks.landmark[f].x * image.shape[1]
        y = results.pose_landmarks.landmark[f].y * image.shape[0]
        if PLOT:
            circle1 = plt.Circle((x, y), size, color=color)
            plt.gca().add_patch(circle1)
    if PLOT:
        plt.imshow(image)
        plt.show()   
    
    orientation = ["front", "right", "back", "left"][np.argmin([a,b,c,d])]
    hipsInPicture = True
    headInPicture = True
    for f in range(32):
        if results.pose_landmarks.landmark[f].y > 1.2: hipsInPicture = False
    for f in range(32):
        if results.pose_landmarks.landmark[f].y < 0.05: headInPicture = False
    
    ys = []
    xs = []
    for f in range(32):
        ys.append(results.pose_landmarks.landmark[f].y * image.shape[0])
        xs.append(results.pose_landmarks.landmark[f].x * image.shape[1])
   
    x1, y1, x2, y2 = int(np.min(xs)), int(np.min(ys)), int(np.max(xs)), int(np.max(ys))
    vv = abs(y1-y2) 
    uu = abs(x2-x1)
    cropareaBody = [int(max(x1 - uu*0.1,0)), 
                    int(max(y1 - vv*0.02,0)), 
                    int(min(x2 + uu*0.1, image.shape[1])), 
                    int(min(y2 + vv*0.02, image.shape[0]))]
    
    return {"orientation": orientation, "lowerlimb_inpicture": hipsInPicture, 
            "head_inpicture": headInPicture, "crop_area_body": cropareaBody}


def extract_face(results, image):
    annotated_image = image.copy()
    if results.multi_face_landmarks is not None:
        for face_landmarks in results.multi_face_landmarks:
            for f in range(468):          
                if(np.isin(f, [10, 137, 152, 366])):
                    color = "red"
                    size = 50
                else:
                    color = "yellow"
                    size = 10
                if PLOT:
                    circle1 = plt.Circle((results.multi_face_landmarks[0].landmark[f].x * image.shape[1], results.multi_face_landmarks[0].landmark[f].y * image.shape[0]), size, color=color)
                    plt.gca().add_patch(circle1)
                    
        _x = results.multi_face_landmarks[0].landmark[10].x * image.shape[1]
        _y = results.multi_face_landmarks[0].landmark[10].y * image.shape[0]
        _x2 = results.multi_face_landmarks[0].landmark[152].x * image.shape[1]
        _y2 = results.multi_face_landmarks[0].landmark[152].y * image.shape[0]
        vector1 = np.array([_x, _y]) - np.array([_x2,_y2])
        if PLOT:
            plt.plot([_x,_x2], [_y,_y2], label="v1")
        
        _x = results.multi_face_landmarks[0].landmark[10].x * image.shape[1]
        _y = results.multi_face_landmarks[0].landmark[10].y * image.shape[0]
        _x2 = results.multi_face_landmarks[0].landmark[137].x * image.shape[1]
        _y2 = results.multi_face_landmarks[0].landmark[137].y * image.shape[0]
        vector2 = np.array([_x, _y]) - np.array([_x2,_y2])
        if PLOT:
            plt.plot([_x,_x2], [_y,_y2], label="v2")
        
        _x = results.multi_face_landmarks[0].landmark[10].x * image.shape[1]
        _y = results.multi_face_landmarks[0].landmark[10].y * image.shape[0]
        _x2 = results.multi_face_landmarks[0].landmark[366].x * image.shape[1]
        _y2 = results.multi_face_landmarks[0].landmark[366].y * image.shape[0]
        vector3 = np.array([_x, _y]) - np.array([_x2,_y2])
        if PLOT:
            plt.plot([_x,_x2], [_y,_y2], label="v3")
        
        
        myDict = {"headTurned": False}
        if min(angle(vector1, vector2), angle(vector1, vector3)) < 0.45:
            myDict["headTurned"] = True
        #print("angle", angle(vector1, vector2), angle(vector1, vector3))
        
        if PLOT:
            #plt.plot([_x,_x2], [_y,_y2])
            plt.imshow(image)
            plt.legend()
            plt.show()
            
            
            
        x1 = (results.multi_face_landmarks[0].landmark[137].x * image.shape[1])
        y1 = (results.multi_face_landmarks[0].landmark[10].y * image.shape[0])
        x2 = (results.multi_face_landmarks[0].landmark[366].x * image.shape[1])
        y2 = (results.multi_face_landmarks[0].landmark[152].y * image.shape[0])
        w = x2-x1
        h = y2-y1
        x1 = np.maximum(0,x1-(w*0.2))
        x2 = np.minimum(image.shape[1],x2+(w*0.2))
        y1 = np.maximum(0,y1-(h*0.3))
        y2 = np.minimum(image.shape[0],y2+(h*0.2))
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        myDict["crop_area_face"] = [x1, y1, x2, y2]  # todo: hier noch augen landmark mit rein
        
        

        return myDict

#https://google.github.io/mediapipe/solutions/pose

def stats_to_label(stats):
    assert type(stats) == dict
    
    if stats["hasBody"] and not stats["hasFace"]: # nur körper
        if stats["lowerlimb_inpicture"]:
            if stats["head_inpicture"] == False:
                if stats["orientation"] == "right":  return "body right"
                elif stats["orientation"] == "front": return "body front"
                elif stats["orientation"] == "back":  return "body back"
                elif stats["orientation"] == "left":  return "body left"
                else: print("error. not impl")
            else:
                print("bend over")
        else:
            return "face side or back"
            
    if stats["hasFace"] and stats["hasBody"]: # gesicht und körper
        if stats["lowerlimb_inpicture"] == True: print("body and face") 
        else:
            if stats["orientation"] == "front": 
                if stats["headTurned"] == False:
                    return "face front"
                else:
                    print("has face, has body, oriented to front, head turned slightly")
            else: print("Error. No implementation for", stats["orientation"])
    
    if stats["hasFace"] and not stats["hasBody"]:
        if stats["headTurned"] == False:
            return "face front"
        else:
            print("Error not implementeddd")


In [ ]:
def pad_to_square(im, fill=0):
    long = np.max(im.shape[0:2])
    short = np.min(im.shape[0:2])
    if len(im.shape) == 3:
        if im.shape[0] < im.shape[1]:
            im=np.pad(im,(((long-short)//2,(long-short)//2),(0,0), (0,0)), constant_values=fill)
        else:
            im = np.pad(im, ((0,0), ((long-short)//2,(long-short)//2), (0,0)), constant_values=fill)
    else:
        if im.shape[0] < im.shape[1]:
            im=np.pad(im,(((long-short)//2,(long-short)//2),(0,0)), constant_values=fill)
        else:
            im = np.pad(im, ((0,0), ((long-short)//2,(long-short)//2)), constant_values=fill)
    return im

In [ ]:
def detect_label(k):
    k = k.replace("\\", "/")
    
    #print("__"*50)
    #print("Processing", k)

    img = cv2.imread(k)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
    if PLOT:
        plt.figure(figsize=(1,1))
        plt.imshow(img)
        plt.axis("off")
        plt.show()
    
    # run mediapipe
    results_facedet = find_head(img)
    results_body = find_body(img)
    
    stats = {"hasBody":  results_body.pose_landmarks is not None, 
             "hasFace": results_facedet.multi_face_landmarks is not None}

    # append face statistics
    if stats["hasFace"]:
        stats_face = extract_face(results_facedet, img)
        for key in stats_face.keys():
            stats[key] = stats_face[key]
    
    # append body statistics
    if stats["hasBody"]:
        stats_body = extract_landmarks(results_body, img)
        for key in stats_body.keys():
            stats[key] = stats_body[key]

    label = stats_to_label(stats)
    print("label:", label)
    #print(stats)
    
    
    
    ######################################################
    ####### safe crops for face and body back/front#######
    ######################################################
    
    patientfolder = SAVE_DIR + "/" + k.split("/")[2] + "/"
    ps = k.split("/")[3]
    def try_make_folder(patientfolder):
        try:
            os.mkdir(patientfolder)
        except:
            print("folder already exists:", patientfolder)
    
    try:
        if "crop_area_face" in stats.keys() and label == "face front":
            try_make_folder(patientfolder)
            x1, y1, x2, y2 = stats["crop_area_face"]  # todo: hier das neue übergeben (siehe oben) und als csv mitsaven
            tosave = pad_to_square(cv2.cvtColor(img[y1:y2, x1:x2], cv2.COLOR_BGR2RGB))
            tosave = cv2.resize(tosave, (1028, 1028))
            cv2.imwrite(patientfolder + f"/face_{ps}.jpg", tosave)
            

        elif label in ["body back"] and "crop_area_body" in stats.keys():
            try_make_folder(patientfolder)
            x1, y1, x2, y2 = stats["crop_area_body"]
            tosave = pad_to_square(cv2.cvtColor(img[y1:y2, x1:x2], cv2.COLOR_BGR2RGB))
            tosave = cv2.resize(tosave, (1028, 1028))
            cv2.imwrite(patientfolder + f"/back_{ps}.jpg", tosave)

        elif label in ["body front"] and "crop_area_body" in stats.keys():
            try_make_folder(patientfolder)
            x1, y1, x2, y2 = stats["crop_area_body"]
            tosave = pad_to_square(cv2.cvtColor(img[y1:y2, x1:x2], cv2.COLOR_BGR2RGB))
            tosave = cv2.resize(tosave, (1028, 1028))
            cv2.imwrite(patientfolder + f"/front_{ps}.jpg", tosave)
    except:
        print("error", k)

    #if PLOT:
    #    if label in ["body front", "body "]
    #    plt.imshow(img[y1:y2, x1:x2])
    #    plt.axis("off")
    #    plt.show()
        
        
    return [k, k.split("/")[2], label]

In [ ]:
from joblib import Parallel, delayed

results = Parallel(n_jobs=16, verbose=0, backend="threading", timeout=10)(
             map(delayed(detect_label), tqdm(all_imgs)))


## Save face keypoint information
#### Only store the facial keypoints for facedrop analyses

In [ ]:
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
import os, glob

In [ ]:
SAVE_DIR = "D:/mediapipe/"

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh
    
def find_head(image):
    with mp_face_mesh.FaceMesh(min_detection_confidence=0.7, min_tracking_confidence=0.5) as face_mesh:
        drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
        
        results = face_mesh.process(image)
    return results

In [ ]:
len(glob.glob(SAVE_DIR+"/*/face*"))

In [ ]:
all_faces_imgs = glob.glob(SAVE_DIR+"/*/face*.jpg")

def save_face_landmark_numpy(pth):
    im = cv2.imread(pth)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    results = find_head(im)
    if results.multi_face_landmarks is None:
        os.remove(pth)
        print("Removed file", pth)
        return
    face_points = np.array([(results.multi_face_landmarks[0].landmark[i].x, 
      results.multi_face_landmarks[0].landmark[i].y) for i in range(468)])
    np.save(pth.split(".")[0]+".npy", face_points)

In [ ]:
from joblib import Parallel, delayed

results = Parallel(n_jobs=16, verbose=0, backend="threading", timeout=10)(
             map(delayed(save_face_landmark_numpy), tqdm(all_faces_imgs)))

In [ ]:
im = cv2.imread(pth)
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
results = find_head(im)

face_points = np.array([(results.multi_face_landmarks[0].landmark[i].x *im.shape[1], 
  results.multi_face_landmarks[0].landmark[i].y *im.shape[0]) for i in range(468)])

In [ ]:
plt.figure(figsize=(14,14))
plt.imshow(im)
plt.scatter(face_points.T[0], face_points.T[1])
for i in range(468):
    plt.annotate(str(i), (face_points.T[0][i], face_points.T[1][i]), color="r", size=8)

In [ ]:
template = np.zeros_like(im)

In [ ]:
eye_right = [107,66,105,63,70,156,35,31,228,229,230,231,232,128,245,193,55]
eye_left = [336, 296, 334, 293, 300, 383, 265, 261, 448, 449, 450, 451, 452, 357, 465, 417, 285]
nase = [151, 336, 285, 417, 465, 343, 277, 355, 429, 358, 327, 326, 2, 97,98,129, 209, 126, 47, 114, 245, 193, 55, 107]
mund = [0,267,269,270,409,287,273,335,406,313,18,83,182,106,43,57,185,40,39,37]

In [ ]:
contours = face_points[eye_right].astype(int)
cv2.fillPoly(template, pts = [contours], color =(255,0,0))

contours = face_points[eye_left].astype(int)
cv2.fillPoly(template, pts = [contours], color =(255,0,0))

contours = face_points[nase].astype(int)
cv2.fillPoly(template, pts = [contours], color =(255,255,0))

contours = face_points[mund].astype(int)
cv2.fillPoly(template, pts = [contours], color =(255,0,255))

plt.imshow(template)